In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv(
    "https://github.com/ulewis/Ejemplos/raw/main/Datos/Classified_Data.txt",
    index_col=0
)

In [3]:
# Clipping 1–99% para estabilizar outliers
num_cols = df.columns.drop('TARGET CLASS')
Q01 = df[num_cols].quantile(0.01)
Q99 = df[num_cols].quantile(0.99)
df[num_cols] = df[num_cols].clip(Q01, Q99, axis=1)

In [4]:
# Escalado sobre TODO el dataset (tal como suelen hacer en estos labs)
scaler = StandardScaler()
X_all = scaler.fit_transform(df.drop("TARGET CLASS", axis=1))
y_all = df["TARGET CLASS"].values

In [5]:
# Baseline: mismo split y k≈24
def eval_split_k(random_state=101, k=24, weights="uniform", p=2):
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all, test_size=0.50, random_state=random_state  # sin stratify, como en el cuaderno
    )
    model = KNeighborsClassifier(n_neighbors=k, weights=weights, p=p)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    err = (1 - acc) * 100
    return err, acc, y_test, y_pred, model

err_base, acc_base, y_test_b, y_pred_b, model_b = eval_split_k(random_state=101, k=24, weights="uniform", p=2)
print("=== BASELINE (clipping + k=24) ===")
print(f"Accuracy : {acc_base*100:.2f}%")
print(f"Tasa de error: {err_base:.2f}%")
print(classification_report(y_test_b, y_pred_b, digits=4))

=== BASELINE (clipping + k=24) ===
Accuracy : 95.40%
Tasa de error: 4.60%
              precision    recall  f1-score   support

           0     0.9522    0.9560    0.9541       250
           1     0.9558    0.9520    0.9539       250

    accuracy                         0.9540       500
   macro avg     0.9540    0.9540    0.9540       500
weighted avg     0.9540    0.9540    0.9540       500

